In [1]:
import pandas as pd

In [2]:
scores = pd.read_csv("conservation_scores.csv")
cpgs = pd.read_csv("methylation_coordinates.csv")

In [3]:
scores


,chromosome,position,score,name
0,chr1,11702,0.544299,chr1.1
1,chr1,11703,0.637228,chr1.1
2,chr1,11704,0.670417,chr1.1
3,chr1,11705,0.816449,chr1.1
4,chr1,11706,0.829724,chr1.1
...,...,...,...,...
15609131,chr10,7881383,0.509961,chr10.29763
15609132,chr10,7882341,0.497843,chr10.29773
15609133,chr10,7894077,0.509110,chr10.29838
15609134,chr10,7894078,0.588850,chr10.29838


In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count

# Define the function to process each chunk of scores
def process_chunk(score_chunk):
    result = pd.DataFrame()
    
    for _, score_row in score_chunk.iterrows():
        # Find matching rows in cpgs based on the specified conditions
        matching_rows = cpgs[
            (cpgs['CHR_hg38'] == score_row['chromosome']) &
            (int(cpgs['Start_hg38']) <= score_row['position']) &
            (int(cpgs['End_hg38']) >= score_row['position'])
        ]
        
        if not matching_rows.empty:
            matching_rows = matching_rows.copy()
            matching_rows['position'] = score_row['position']
            matching_rows['chromosome'] = score_row['chromosome']
            matching_rows['score'] = score_row['score']
            
            # Define the column order
            ordered_columns = ['IlmnID', 'Name', 'chromosome', 'position', 'score'] + \
                              list(matching_rows.columns.difference(['IlmnID', 'Name', 'chromosome', 'position', 'score']))
            
            # Append matching rows to the result DataFrame with the specified column order
            result = pd.concat([result, matching_rows[ordered_columns]])

    return result

# Main function to set up multiprocessing
if __name__ == '__main__':
    # Number of cores to use
    num_cores = cpu_count()

    # Split scores into chunks for each process
    scores_chunks = np.array_split(scores, num_cores)

    # Create a Pool and map the chunks to the process_chunk function
    with Pool(num_cores) as pool:
        results = pool.map(process_chunk, scores_chunks)
    
    # Combine all results into a single DataFrame
    filtered_cpgs = pd.concat(results).reset_index(drop=True)

    # Display the first few rows of the final DataFrame
    print(filtered_cpgs.head())


c:\Users\MEDV3721\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
